In [1]:
# nlp book

In [2]:
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
tfds.__version__

'4.2.0'

In [4]:
# train_ds, test_ds = dataset['train'], dataset['test']
# train_ds.element_spec


train_ds, ds_info = tfds.load(name="imdb_reviews", split="train", with_info=True, as_supervised=True)
test_ds = tfds.load(name="imdb_reviews", split="test", as_supervised=True)

In [5]:
tokenizer = tfds.deprecated.text.Tokenizer()

VOCABULARY_SET = set()
MAX_TOKEN = 0

for example, label in train_ds:
    some_token = tokenizer.tokenize(example.numpy())
    if MAX_TOKEN < len(some_token):
        MAX_TOKEN = len(some_token)
    VOCABULARY_SET.update(some_token)

In [6]:
for example, label in train_ds.take(1):
    print(example)
    print()
    print(label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)

tf.Tensor(0, shape=(), dtype=int64)


In [7]:
imdb_encoder = tfds.deprecated.text.TokenTextEncoder(VOCABULARY_SET, tokenizer=tokenizer)
vocab_size = imdb_encoder.vocab_size
print(vocab_size, MAX_TOKEN)

93931 2525


In [8]:
for example, label in train_ds.take(1):
    print(example)
    encoder = imdb_encoder.encode(example.numpy())
    print(encoder)
    decoder = imdb_encoder.decode(encoder)
    print(decoder)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
[571, 91863, 92156, 43263, 57079, 80964, 43877, 42159, 33167, 61314, 31028, 82917, 72879, 82963, 66734, 3750, 39469, 21705, 80776, 60841, 84040, 1653, 5657, 56036, 9299, 33167, 70253, 93281, 72918, 31028, 85589, 27938, 70253, 60841, 58377, 32752, 24187

In [18]:
def encode_pad_transform(sample):
    encoded = imdb_encoder.encode(sample.numpy())
    pad = pad_sequences(['encoded'], padding='post', maxlen=150, truncating='post')
    return np.array(pad[0], dtype=np.int64)


def encode_tf_fn(sample, label):
    encoded = tf.py_function(encode_pad_transform, inp=[sample], Tout=(tf.int64))
    encoded.set_shape([None])
    label.set_shape([])
    return encoded, label

In [21]:
subset = train_ds.take(10)
tst = subset.map(encode_tf_fn)
for example, label in tst.take(1):
    print(example, label)
    print(imdb_encoder.decode(review))

InvalidArgumentError: ValueError: invalid literal for int() with base 10: 'encoded'
Traceback (most recent call last):

  File "/home/tim/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 241, in __call__
    return func(device, token, args)

  File "/home/tim/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 130, in __call__
    ret = self._func(*args)

  File "/home/tim/anaconda3/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 309, in wrapper
    return func(*args, **kwargs)

  File "<ipython-input-18-6e9ba8bd56c2>", line 3, in encode_pad_transform
    pad = pad_sequences(['encoded'], padding='post', maxlen=150, truncating='post')

  File "/home/tim/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/preprocessing/sequence.py", line 156, in pad_sequences
    return sequence.pad_sequences(

  File "/home/tim/.local/lib/python3.8/site-packages/keras_preprocessing/sequence.py", line 98, in

In [ ]:
encoded_train=(train_ds.map(encode_tf_fn))

In [ ]:
#for review, label in encoded_train.take(1):
 #   print(review)